# Desafío - Inferencia de tópicos con EM

Luis Porras

### Descripción

* En esta sesión trabajaremos con una serie de base de datos sobre letras musicales de distintos artistas. Cada uno de los csv se encuentra en la carpeta `dump`.
* Cada csv tiene el nombre del artista a analizar. Los archivos contienen el nombre del artista, el género musical del artista, el nombre de la canción y las letras.
* En base a esta información, el objetivo del ejercicio es generar un modelo probabilístico que pueda identificar el género musical más probable dado la letra de una canción. Para ello implementaremos un modelo conocido como Latent Dirichlet Allocation que hace uso de una variante del algoritmo EM para inferir clases latentes a partir de una matriz de documentos.

In [1]:
# Librerías básicas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import glob
import os

warnings.filterwarnings(action='ignore')

# Configuración de gráficas
plt.rcParams["figure.figsize"] = (20, 15)
plt.rcParams["figure.dpi"] = 80
sns.set_style('darkgrid')

from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.decomposition import LatentDirichletAllocation # Variación del algoritmo EM para extracción de tópicos

In [2]:
pd.read_csv(os.getcwd() +  '/' + glob.glob('dump/*.csv')[0]).head()

,Unnamed: 0,0,1,2,3
0,0,Public Enemy,hiphop,You're Gonna Get Yours,"(Flavor Flav) \n Oh-oh Chuck, they out to get ..."
1,1,Public Enemy,hiphop,Sophisticated Bitch,"That woman in the corner, cold playin' the rol..."
2,2,Public Enemy,hiphop,Miuzi Weighs A Ton,"Yo Chuck, run a power move on them \n (Yeah) \..."
3,3,Public Enemy,hiphop,Timebomb,"(Intro - Flavor Flav) \n Hey, Chuck, we got so..."
4,4,Public Enemy,hiphop,Too Much Posse,(Intro - Flavor Flav) \n What do you got to sa...


In [3]:
# Recuperar la Lista con todos los .csv en la carpeta dump
current_path = os.getcwd()
dump_dfs = list(map(lambda path: pd.read_csv(current_path + '/' + path), glob.glob('dump/*.csv')))
df = pd.concat(dump_dfs).drop(columns=['Unnamed: 0'])
df.columns = ['Artist', 'Genre', 'Song', 'Lyrics']
df.sample(5)

,Artist,Genre,Song,Lyrics
100,Queen,rock,Back Chat,Words and music by John Deacon \n Yeah \n Get ...
143,Kiss,rock,Turn On The Night,Everyone's thinkin' they're so hot tonight \n ...
391,Eminem,hiphop,Nuttin To Do,What? Uh \n The bad \n Yeah \n The Evil \n Rig...
5,Black Star,hiphop,Brown Skin Lady,"(Mos Def) \n Be like yo, let me ask you someth..."
85,SIA,pop,Move Your Body,Poetry on your body \n You got it in every way...


In [4]:
df.shape

(9489, 4)

In [5]:
df['Genre'].value_counts()

rock      4140
hiphop    2535
metal     1582
pop       1232
Name: Genre, dtype: int64

In [6]:
counter = CountVectorizer(stop_words='english', max_features=2500, max_df=.1)

transformed_feats = counter.fit_transform(df['Lyrics'])

transformed_feats.toarray().shape

(9489, 2500)

In [ ]:
search_params = {'n_components': [5, 10, 15], 'learning_decay': [0.7, 0.5]}

In [10]:
%%time
cv_lda_model = GridSearchCV(LatentDirichletAllocation(learning_method='online'),
                            param_grid=search_params, n_jobs=-1,
                           cv=5).fit(transformed_feats)

KeyboardInterrupt: 

In [9]:
best_lda = cv_lda_model.best_estimator_
best_lda

NameError: name 'cv_lda_model' is not defined